# MODELADO

## Dense Newtork


<hr>

<code> **Proyecto de Datos II** </code>

## Índice

- [Importación de los datos](#importación-de-los-datos)
- [Preprocesamiento](#preprocesamiento)
- [Entrenamiento](#entrenamiento)
- [Análisis del modelo](#análisis-del-modelo)
- [Registro del modelo en MLflow](#registro-del-modelo-en-mlflow)


In [3]:
import time
import mlflow
import pandas as pd
from evaluation.evaluator import Evaluator

SEED = 22 # replicabilidad

# =====================================
MODEL_NAME = "dense_network_by_runway"
# =====================================

## Importación de los datos

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import keras_tuner as kt

# Cargar datos
df_train = pd.read_parquet("../data/train.parquet")
df_test = pd.read_parquet("../data/test.parquet")

# Separar características y variable objetivo
X_train_full = df_train.drop(columns="takeoff_time")
y_train_full = df_train["takeoff_time"]

X_test = df_test.drop(columns="takeoff_time")
y_test = df_test["takeoff_time"]

# Convertir columna timestamp a datetime
X_train_full["timestamp"] = pd.to_datetime(X_train_full["timestamp"])

# Dividir entre train y validation basado en fecha
split_date = pd.to_datetime("2025-01-01")

train_mask = X_train_full["timestamp"] < split_date
val_mask = X_train_full["timestamp"] >= split_date

X_train = X_train_full[train_mask].drop(columns=["timestamp", "icao", "callsign"])  # Puedes ajustar qué columnas eliminar
y_train = y_train_full[train_mask]

X_val = X_train_full[val_mask].drop(columns=["timestamp", "icao", "callsign"])
y_val = y_train_full[val_mask]

# Procesar X_test también (eliminar columnas que no se usan)
X_test = X_test.drop(columns=["timestamp", "icao", "callsign"])

In [6]:
X_train.shape, X_val.shape, X_test.shape

((98645, 57), (25088, 57), (27791, 57))

## Preprocesamiento

In [8]:
# One-hot encoding

X_train_c = X_train
X_val_c = X_val
X_test_c = X_test

X_train = pd.get_dummies(X_train, drop_first=True)
X_val = pd.get_dummies(X_val, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Alinear columnas por si falta alguna categoría
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## Entrenamiento

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import keras_tuner as kt
import time

start_time = time.time()

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import keras_tuner as kt

# 6. Definición de modelo para Keras Tuner con Bayesian Optimization
def build_model(hp, input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))

    # Número de capas ocultas
    num_layers = hp.Int('num_layers', 1, 3)  # 1, 2 o 3 capas ocultas
    for i in range(num_layers):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),  # Unidades entre 64 y 512
            activation='relu'
        ))
        
        # Agregar capa Dropout
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))  # Dropout entre 0.2 y 0.5

    model.add(layers.Dense(1))  # Salida para regresión

    # Compilación con optimizador Adam y búsqueda del learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-5, 1e-2, sampling='LOG'),
            decay=hp.Float('lr_decay', 1e-6, 1e-3, sampling='LOG')  # Decaimiento del learning rate
        ),
        loss='mse',
        metrics=['mae']
    )
    return model

# Definir búsqueda con Bayesian Optimization
def tune_model(X_train_scaled, y_train, X_val_scaled, y_val):
    tuner = kt.BayesianOptimization(
        lambda hp: build_model(hp, X_train_scaled.shape[1]),
        objective='val_mae',
        max_trials=20,  # Número total de pruebas
        directory='kt_tuning_dir_bayesian',
        project_name='takeoff_time',
        overwrite=True  # Sobrescribir cualquier proyecto anterior
    )

    early_stop = callbacks.EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)

    tuner.search(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=30,
        batch_size=32,
        callbacks=[early_stop],
        verbose=2
    )

    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_hp, tuner

# Crear y entrenar modelos para cada pista
def train_models_per_runway(X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test):
    model_results = {}

    for runway_value in df_train['runway'].unique():
        print(f"\nEntrenando modelo para runway = {runway_value}")
        
        train_indices = X_train_c["runway"] == runway_value
        val_indices = X_val_c["runway"] == runway_value
        test_indices = X_test_c["runway"] == runway_value
        
        # Filtrar las características (X) y las etiquetas (y) según los índices
        X_train_runway = X_train_scaled[train_indices]
        y_train_runway = y_train[train_indices]
        
        X_val_runway = X_val_scaled[val_indices]
        y_val_runway = y_val[val_indices]
        
        X_test_runway = X_test_scaled[test_indices]
        y_test_runway = y_test[test_indices]

        # 8. Ejecutar tuning para este modelo específico
        print("\nBuscando mejores hiperparámetros...")
        best_hp, tuner = tune_model(X_train_runway, y_train_runway, X_val_scaled, y_val)

        # 9. Entrenar modelo final con los mejores hiperparámetros
        print(f"\nEntrenando modelo final para runway = {runway_value} con mejores hiperparámetros...")
        final_model = build_model(best_hp, X_train_runway.shape[1])

        early_stop_final = callbacks.EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)

        history = final_model.fit(
            X_train_runway, y_train_runway,
            validation_data=(X_val_scaled, y_val),
            epochs=100,
            batch_size=32,
            callbacks=[early_stop_final],
            verbose=2
        )

        # Evaluación
        test_loss, test_mae = final_model.evaluate(X_test_runway, y_test_runway, verbose=0)
        print(f"Test MAE para runway {runway_value}: {test_mae:.2f}")

        # Almacenar resultados
        model_results[runway_value] = {
            'model': final_model,
            'mae': test_mae,
            'history': history.history,
        }

    return model_results

# Llamar a la función para entrenar modelos por cada pista
model_results = train_models_per_runway(X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test)

end_time = time.time()
execution_time = end_time - start_time


Trial 20 Complete [00h 00m 20s]
val_mae: 126.67001342773438

Best val_mae So Far: 120.95462036132812
Total elapsed time: 00h 04m 50s

Entrenando modelo final para runway = 32L/14R con mejores hiperparámetros...
Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


58/58 - 1s - 20ms/step - loss: 31050.8711 - mae: 129.4714 - val_loss: 37389.9766 - val_mae: 146.3074
Epoch 2/100
58/58 - 1s - 13ms/step - loss: 6383.1870 - mae: 59.6084 - val_loss: 36131.6875 - val_mae: 140.4044
Epoch 3/100
58/58 - 1s - 13ms/step - loss: 3945.2896 - mae: 46.3159 - val_loss: 34379.0039 - val_mae: 131.9752
Epoch 4/100
58/58 - 1s - 13ms/step - loss: 2496.0610 - mae: 36.9849 - val_loss: 33522.8789 - val_mae: 126.5166
Epoch 5/100
58/58 - 1s - 13ms/step - loss: 1717.6821 - mae: 30.2045 - val_loss: 33463.8477 - val_mae: 122.9222
Epoch 6/100
58/58 - 1s - 13ms/step - loss: 1278.5308 - mae: 25.8570 - val_loss: 34088.0156 - val_mae: 121.5498
Epoch 7/100
58/58 - 1s - 13ms/step - loss: 1039.0775 - mae: 23.3517 - val_loss: 35040.4023 - val_mae: 121.9139
Epoch 8/100
58/58 - 1s - 13ms/step - loss: 851.7244 - mae: 21.1158 - val_loss: 35782.4141 - val_mae: 122.1730
Epoch 9/100
58/58 - 1s - 13ms/step - loss: 856.1193 - mae: 21.1830 - val_loss: 36214.1836 - val_mae: 122.2297
Epoch 10/100


## Análisis del modelo

In [12]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Inicializamos listas para almacenar los MAE y RMSE de cada modelo
mae_train_list = []
rmse_train_list = []

mae_val_list = []
rmse_val_list = []

# Recorremos los resultados de cada runway
for runway_value, result in model_results.items():
    model = result['model']
    
    # Predicciones en el conjunto de entrenamiento
    y_train_pred = model.predict(X_train_scaled)  # Predicciones sobre X_train_scaled
    mae_train = mean_absolute_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    # Predicciones en el conjunto de validación
    y_val_pred = model.predict(X_val_scaled)  # Predicciones sobre X_val_scaled
    mae_val = mean_absolute_error(y_val, y_val_pred)
    rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    # Almacenamos los valores
    mae_train_list.append(mae_train)
    rmse_train_list.append(rmse_train)
    
    mae_val_list.append(mae_val)
    rmse_val_list.append(rmse_val)

# Calculamos la media de los MAE y RMSE en entrenamiento y validación
mae_train= np.mean(mae_train_list)
rmse_train = np.mean(rmse_train_list)

mae_val = np.mean(mae_val_list)
rmse_val = np.mean(rmse_val_list)

# Mostramos los resultados
print(f"MAE promedio en entrenamiento: {mae_train:.4f}")
print(f"RMSE promedio en entrenamiento: {rmse_train:.4f}")
print(f"MAE promedio en validación: {mae_val:.4f}")
print(f"RMSE promedio en validación: {rmse_val:.4f}")


3083/3083 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step
784/784 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step
3083/3083 ━━━━━━━━━━━━━━━━━━━━ 1s 339us/step
784/784 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
3083/3083 ━━━━━━━━━━━━━━━━━━━━ 2s 662us/step
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step
3083/3083 ━━━━━━━━━━━━━━━━━━━━ 2s 602us/step
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step
MAE promedio en entrenamiento: 87.3972
RMSE promedio en entrenamiento: 130.6235
MAE promedio en validación: 105.8320
RMSE promedio en validación: 162.9917


In [13]:
# Inicializar una columna de predicciones en df_test
df_test['prediction'] = np.nan

# Iterar sobre cada modelo y su correspondiente 'runway'
for runway_value, result in model_results.items():
    model = result['model']
    runway_data = df_test[df_test['runway'] == runway_value].copy()  # Filtrar datos por runway

    if runway_data.empty:
        continue  # Si no hay datos para este runway, omitir

    test_indices = X_test_c["runway"] == runway_value

    X_test_runway = X_test_scaled[test_indices]
    y_test_runway = y_test[test_indices]

    # Hacer predicciones usando el modelo correspondiente
    y_runway_pred = model.predict(X_test_runway)

    # Asignar las predicciones al DataFrame original de prueba
    df_test.loc[df_test['runway'] == runway_value, 'prediction'] = y_runway_pred


315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step


In [14]:
# Nota: df_test tiene que tener la columna 'prediction'
ev = Evaluator(df_test, MODEL_NAME, mae_val, rmse_val)
report = ev.getReport()
ev.visualEvaluation()

## Registro del modelo en MLflow

In [16]:
mlflow.set_tracking_uri("file:./mlflow_experiments")
mlflow.set_experiment("takeoff_time_prediction")

with mlflow.start_run():

    # - Datos generales -

    # ========================================================================
    mlflow.set_tag("model_type", MODEL_NAME)
    mlflow.set_tag("framework", "tensorflow") # scikit-learn, tensorflow, etc.
    mlflow.set_tag("target_variable", "takeoff_time") # variable respuesta
    mlflow.set_tag("preprocessing", "standard scaler") # transformaciones separadas por un +
    mlflow.set_tag("dataset", "original") # indicar si se ha modificado el conjunto de datos
    mlflow.set_tag("seed", SEED) # semilla para replicabilidad
    # ========================================================================
    

    # - Métricas -

    mlflow.log_metric("execution_time_s", execution_time)

    mlflow.log_metric("mae_val", mae_val)
    mlflow.log_metric("rmse_val", rmse_val)

    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("rmse_train", rmse_train)

    # Registrar métricas globales en test
    for metric_name, value in report["global"].items():
        mlflow.log_metric(f"{metric_name}_test", value)
    
    # Registrar métricas por runway
    for runway, metrics in report["by_runway"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_runway_{runway}", value)
    
    # Registrar métricas por holding point
    for hp, metrics in report["by_holding_point"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_hp_{hp}", value)

    for runway_value, result in model_results.items():
        model = result['model']
    
        # Guardar los hiperparámetros de este modelo
        if 'best_hp' in locals():
            best_hp_dict_runway = {key: value for key, value in best_hp.items()}  # Convertir a un diccionario
            # Agregar el runway_value al nombre de los parámetros
            mlflow.log_params({f"{key}_runway_{runway_value}": value for key, value in best_hp_dict_runway.items()})
        
        # Guardar el modelo de cada runway
        mlflow.keras.log_model(model, f"model_runway_{runway_value}")
    

2025/04/27 00:46:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/27 00:46:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/27 00:46:51 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/27 00:46:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/27 00:46:57 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/27 00:47:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/27 00:47:02 WARNING mlflow.keras.save: You are saving a Keras model 

In [17]:
# - Visualizar experimentos -
# !mlflow ui --backend-store-uri ./mlflow_experiments